In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.webdriver import FirefoxBinary
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from pyquery import PyQuery as pq
import re
import csv
import time
import random

In [2]:
#保存csv
def save_csv(hometown,append_country,append_city,name_list,url_of_members,gender_list,append_city_time,append_other_country_time,name):
    print("save_csv")
    data_all = []   
    index = 0
    for num in range(0,len(hometown)):
        data = [] 
        data.append(index)
        data.append(url_of_members[index])
        data.append(name_list[index])
        data.append(gender_list[index])
        data.append(hometown[index])
        data.append(append_country[index])
        data.append(append_other_country_time[index])
        data.append(append_city[index])
        data.append(append_city_time[index])
        data.append(len(append_country[index]))
        data.append(len(append_city[index]))
        print(append_city[index])
        index = index +1
        data_all.append(data)

    out = open('mfw_result_final2018.csv','w', newline='')
        #设定写入模式
    csv_write = csv.writer(out,dialect='excel')
        #写入具体内容
    for a in data_all:
        print(a)
        csv_write.writerow(a)

In [3]:
    #3. 重新打开网页进行抓取
def sub_craw(driver,url_of_members,name_list):
        sum = 1 
        gender = ''
        gender_list = []
        #设置存储数据结构
        append_country = []
        append_city = []
        city_time = []
        append_city_time = []
        append_other_country_time = []
        hometown=[]
        city = ''
        interation = 0
        for sub_url in url_of_members:
            interation = interation + 1
            if(interation % 20 == 0):
                time.sleep(random.randint(0,10))
            if(interation % 50 == 0):
                time.sleep(random.randint(0,15))
            # 每次循环写一趟算了!!!!!!!!!
            print(sum,'.  ','crawing sub_url :',sub_url)
            sum = sum + 1
            
            time.sleep(0.5)
            try:
               
                time.sleep(random.randint(0,3))
                driver.get(sub_url)
                time.sleep(1)
            except Exception as e:  
                #change here
                
                print('url:   ',e)
                continue
                #得到居住地
            #1.爬居住地
            try:
                print(interation)
                b = driver.find_element_by_class_name('MAvaPlace')
                a = b.text.split('：')
                print('     ','home town:  ',a[1])
                hometown.append(a[1])
            except Exception as e:  
                hometown.append('')
                #2.爬性别
            try:
                print(interation)
                b = driver.find_element_by_class_name('MAvaName').find_element_by_xpath("i").get_attribute("class") 
                print('name',b)  
                gender = b
                gender_list.append(gender)
            except Exception as e:  
                print(e)
                gender = 'null'
                gender_list.append(gender)
                #切换栏目     
            try:
                select_area = WebDriverWait(driver, 20).until(
                EC.visibility_of(driver.find_element_by_partial_link_text('TA的足迹')))
                select_area.click()
            except Exception as e: 
                print(e)
            i = 0
            time.sleep(0.2)    
                #往下滚    
            try:
                while i<10:
                    i = i+1
                    time.sleep(0.01)
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            #跳转到足迹准备爬
            except Exception as e: 
                print(e)
            #2nd for 设置国家缓存
            find_china = 0
            china_city = []         #(3)
            china_city_time = []
            other_country = []  #(4)
            other_country_time = []
 
    #b. 给country 添加数据 (国家)
            try:
                sub_sum = 0
                for session in driver.find_elements_by_class_name('country-block'):
                    a = session.find_element_by_class_name("en")
                    b = a.text
                    c = session.find_element_by_class_name("_j_time")
                    d = c.text
                    sub_sum = sub_sum + 1
                    city = b
                    if (city != 'China'):
                        other_country.append(city)
                        other_country_time.append(d)
                        print('  (',sub_sum,')  ' ,city)
                        print("_j_time:   ",d)
                append_other_country_time.append(other_country_time)                     
                append_country.append(other_country)
                other_country = []
                other_country_time = []
            except Exception as e:
                print("国家:     ",e)
                append_other_country_time.append(other_country_time)
                append_country.append(other_country)
                other_country = []
                other_country_time = []

     #3rd for 设置城市缓存           
            try:
                china_city = []
                for session in driver.find_elements_by_class_name('country-block'):
                    a = session.find_element_by_class_name("en")
                    b = a.text
                    if (b == 'China'):
                        sum_city = 0
                        for a in session.find_elements_by_class_name('_j_cityitem'):
                            sum_city = sum_city + 1
                            a1 = a.find_element_by_class_name("_j_time")
                            b = a1.text
                            print('_j_time city: ',b)
                            c = a.find_element_by_class_name('txt')
                            d = c.text.split('\n')
                            print ('       [',sum_city,']    ',d[0])
                            china_city.append(d[0])
                            city_time.append(b)
                
                append_city.append(china_city) 
                append_city_time.append(city_time)
                china_city = []
                city_time = []                
            except Exception as e: 
                print('country  :\n',e)
                print(china_city)
                append_city.append(china_city) 
                append_city_time.append(city_time)
                china_city = []
                city_time = []

            print(len(hometown)," ",len(gender_list),"  ",len(append_country),"  ",len(append_city),len(append_city_time),len(append_other_country_time))                
        save_csv(hometown,append_country,append_city,name_list,url_of_members,gender_list,append_city_time,append_other_country_time,name)

In [ ]:
#从csv读链接:
total_doc =  []
name_list = []

# chromeOptions = webdriver.ChromeOptions()
# chromeOptions.add_argument("--proxy-server=http://112.16.154.190")
# # 一定要注意，=两边不能有空格，不能是这样--proxy-server = http://202.20.16.82:10152
# driver = webdriver.Chrome(options = chromeOptions)

driver = webdriver.Chrome()
driver.set_page_load_timeout(20)
small  = 0
big = 600
with open('mafengwo_url.csv') as csvfile:
    reader = csv.reader(csvfile)
    column = [row[1] for row in reader]
    count = 0
    for a in column:
        count =  count + 1
        if(small-1<count < big and a != "Url"):
#         if(1):
            total_doc.append(a)
with open('mafengwo_url.csv') as csvfile:
    reader = csv.reader(csvfile)
    name = [row[0] for row in reader]
    count = 0
    for a in name:
        count =  count + 1
        if(small-1<count < big and a != "Name"):
#     if(1):
            name_list.append(a)
    print(len(name_list))
    print(len(total_doc)) 
sub_craw(driver,total_doc,name_list) 